In [1]:
import sys
import os

from nlp_datasets import YahooDataset
from nlp_datasets import BaseDataset
from nlp_modeltrainers import BaseTrainerModule
from nlp_modeltrainers.sentence_classification import MulticlassSentenceClassificationTrainerModule
# from nlp_modeltrainers import VectorCosineSimilarityTrainerModule


import torch
import fastwer
from string import ascii_letters as letters
L = list(letters)
import numpy as np
import pytorch_lightning as pl
from pytorch_lightning.callbacks.early_stopping import EarlyStopping

from tqdm import tqdm
from torch.nn import Module, Linear, Embedding
from torch.utils.data import DataLoader

from nltk import word_tokenize

import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset
from pytorch_lightning import LightningModule

dev = 'cpu'
if torch.cuda.is_available():
    dev = "cuda:0"

device = torch.device(dev)

In [2]:
from utils.SubwordHash import subwordhash
from utils.SubwordEmbedding import subwordembedding
from utils.config import *

In [3]:
class Hash_Preprocessor:
    def __init__(self):
        pass
    
    def __call__(self, sample):
        tokenized = word_tokenize(sample["input"])
        tokenized_hashes = self.hash_tokenize(tokenized)
        output_id = self.padding(tokenized_hashes, padding_idx=0)
        
        return {"input": output_id, "target": sample['target']-1}
    
    def hash_tokenize(self, data):
        tokenized_id = [subword_hashes(w) for w in data]
        return tokenized_id
    
    def padding(self, data, padding_idx=0):
        if len(data) >= max_sample_len:
            return torch.tensor(data[:max_sample_len], dtype = torch.long).to(device)
        data.extend(np.array([[padding_idx]*max_sw_hash_len]*(max_sample_len - len(data))))
        return torch.tensor(data, dtype = torch.long).to(device)

In [4]:
class Misspell_Preprocessor:
    def __init__(self):
        pass
    
    def __call__(self, sample):
        random_freq = True
        tokenized = {i:w for i,w in enumerate(word_tokenize(sample["input"]))}
        tokenized = self.misspell(tokenized, random_freq)
        tokenized_hashes = self.hash_tokenize(tokenized)
        output_id = self.padding(tokenized_hashes, padding_idx=0)
        
        return {"input": output_id, "target": sample['target']-1}
    
    def misspell(self, data, random_freq=False):
        if random_freq: 
            msp_f = np.random.uniform(0.1,0.5)
        else: 
            msp_f = misspell_freq
        misspell_num = int(len(data)*msp_f)
        misspell_idx = np.random.choice(len(data), misspell_num, replace = False)
        m_type = {i:mt for i,mt in enumerate(np.random.randint(0, 4, misspell_num))}
        m_dict = {0:self.delete, 1:self.insert, 2:self.replace, 3:self.swap}
        for i in range(misspell_num):
            mp = data[misspell_idx[i]]
            if len(mp) > 1:
                mp = m_dict[m_type[i]](list(mp))
            else:
                mp = self.replace(list(mp))
            data[misspell_idx[i]] = mp
        return [data[w] for w in sorted(data)]
    
    def delete(self, word):
        idx = np.random.randint(len(word))
        word.pop(idx)
        return ''.join(map(str,word))

    def insert(self, word):
        idx = np.random.randint(len(word))
        letter = np.random.choice(L)
        word.insert(idx, letter)
        return ''.join(map(str,word))

    def replace(self, word):
        idx = np.random.randint(len(word))
        letter = np.random.choice(L)
        word.pop(idx)
        word.insert(idx,letter)
        return ''.join(map(str,word))

    def swap(self, word):
        idx1 = np.random.randint(len(word))
        if idx1 == 0:
            idx2 = idx1 + 1
        elif idx1 == len(word)-1:
            idx2 = idx1 -1
        else:
            idx2 = np.random.choice([idx1+1,idx1-1])
        first_idx = min(idx1,idx2)
        second_idx = max(idx1,idx2)
        temp = word.pop(first_idx)
        word.insert(second_idx, temp)
        return ''.join(map(str,word))
        
    def hash_tokenize(self, data):
        tokenized_id = [subword_hashes(w) for w in data]
        return tokenized_id
    
    def padding(self, data, padding_idx=0):
        if len(data) >= max_sample_len:
            return torch.tensor(data[:max_sample_len], dtype = torch.long).to(device)
        data.extend(np.array([[padding_idx]*max_sw_hash_len]*(max_sample_len - len(data))))
        return torch.tensor(data, dtype = torch.long).to(device)

In [5]:
class YahooClassifier(Module):
    def __init__(self, word_embedding, embedding_dim, class_zize,device):
        super().__init__()
        self.device = device
        self.word_embedding = word_embedding.to(device)
        self.linear_classifier = Linear(embedding_dim, class_size).to(device)
    
    def forward(self, token_ids):
        """
        toekn_ids: (batch_size, worrd_num, hash_size)
        """
        debug = False
        if debug: print(token_ids.shape)
        # (batch_size, words_num, embedding_dim)
        outputs = self.word_embedding(token_ids).to(self.device)
        if debug: print(outputs.shape)
        # (batch_size, embedding_dim)
        outputs = torch.max(outputs, dim=1)[0].to(self.device)
        if debug: print(outputs.shape)
        # (batch_size, class_size)
        outputs = self.linear_classifier(outputs).to(self.device)
        if debug: print(outputs.shape)
        return outputs

In [6]:
dataset = YahooDataset(max_samples=max_samples, local_dir="../datasets/small_yahoo_dataset")
subword_hashes = subwordhash(dataset.train)

word_num = subword_hashes.word_num
max_sw_hash_len = subword_hashes.max_hash
max_sample_len = subword_hashes.max_sample
print(max_sw_hash_len)
print(max_sample_len)

100%|█████████████████████████████████████████████████████████████████████████████████| 9000/9000 [00:06<00:00, 1300.83it/s]


377
995


In [7]:
emb_path = emb_path+'_latesum'
word_embedding = subwordembedding(num_embeddings = num_emb, 
                                  embedding_dim = emb_dim, 
                                  device = device, 
                                  padding_idx = 0,
                                  sumfirst = True)
word_embedding = word_embedding.to(device)
word_embedding.load_state_dict(torch.load(emb_path))
print(f'Loaded model: {emb_path}')

Loaded model: ../Trained_Models/SubwordEmbedding/trained_model/trained_model_50d_uniform_100e_3w_latesum


In [8]:
preprocessor = Hash_Preprocessor()

dataset.train.set_preprocessor(preprocessor)
dataset.val.set_preprocessor(preprocessor)
dataset.test.set_preprocessor(preprocessor)

In [9]:
dataloader_train = DataLoader(dataset.train, batch_size=batch_size, shuffle=True)
dataloader_val = DataLoader(dataset.val, batch_size=batch_size, shuffle=False)
dataloader_test = DataLoader(dataset.test, batch_size=batch_size, shuffle=False)

In [10]:
misspell_dataset = YahooDataset(max_samples=max_samples, local_dir="../datasets/misspell_dataset")

In [11]:
preprocessor = Misspell_Preprocessor()

misspell_dataset.train.set_preprocessor(preprocessor)
misspell_dataset.val.set_preprocessor(preprocessor)
misspell_dataset.test.set_preprocessor(preprocessor)

In [12]:
msploader_train = DataLoader(misspell_dataset.train, batch_size=batch_size, shuffle=True)
msploader_val = DataLoader(misspell_dataset.val, batch_size=batch_size, shuffle=False)
msploader_test = DataLoader(misspell_dataset.test, batch_size=batch_size, shuffle=False)

In [13]:
yahoo_classifier = YahooClassifier(word_embedding, emb_dim, class_size,device).to(device)

In [14]:
for batch in dataloader_train:
    outputs = yahoo_classifier(batch["input"])
    print(outputs.shape)
    print(outputs)
    break

torch.Size([8, 10])
tensor([[ -0.9442,   0.1966,  -4.5950,  -2.9982,   4.1343,   0.9446,  -2.8046,
          -0.6334,  -3.0932,   1.0603],
        [ -0.1649,  -3.5567,  -4.5437,  -4.9015,   5.2856,   0.4494,  -5.8076,
          -0.5940,  -3.7902,  -0.6011],
        [ -1.9615,  -2.9647,  -2.9552,  -4.6240,   4.6597,   0.2992,  -5.1500,
           0.0890,  -3.9151,  -0.2487],
        [ -2.9134,  -3.6512,  -6.3358,  -5.8849,   7.3315,  -1.0947,  -8.4590,
           0.8159,  -5.4828,  -1.8113],
        [ -1.2788,  -0.7232,  -4.4302,  -6.5334,   8.5128,   2.1744,  -3.6169,
           1.4120,  -3.7297,  -1.8897],
        [  0.1268,  -3.7274,  -7.9826,  -5.5366,   8.2859,   1.3688,  -6.0273,
           0.8213,  -5.4673,   0.9888],
        [  1.1727,  -3.8323, -12.2453,  -8.1012,  11.3016,   1.4681, -11.2397,
           3.7897, -10.1062,  -1.3480],
        [ -1.3928,  -1.2304,  -4.4864,  -6.6695,   3.9634,   0.2022,  -4.8462,
           0.5809,  -1.8371,  -1.7197]], device='cuda:0',
       gra

In [15]:
classifier_model = MulticlassSentenceClassificationTrainerModule(yahoo_classifier).to(device)

In [16]:
misspell = False
if misspell:
    logger = pl.loggers.CSVLogger("../Trained_Models/Classification/logs", name = f"MisspellText_{emb_dim}d")
else:
    logger = pl.loggers.CSVLogger("../Trained_Models/Classification/logs", name = f"CorrectedText_{emb_dim}d")
    
checkpoint = pl.callbacks.ModelCheckpoint(
    dirpath = "../Trained_Models/Classification/checkpoints",
    filename = 'best_model_{epoch}_{val_loss:.2f}_latesum',
    monitor = 'val_loss',
    mode = 'min'
)
class LitProgressBar(pl.callbacks.ProgressBar):
    def init_validation_tqdm(self):
        bar = tqdm(disable=True)
        return bar
bar = LitProgressBar()

In [17]:
mode = 'cor'
loader = {
    'msp':{'train':msploader_train, 'val':msploader_val, 'test':msploader_test},
    'cor':{'train':dataloader_train, 'val':dataloader_val, 'test':dataloader_test}
}
torch.cuda.empty_cache()
trainer = pl.Trainer(logger = logger, 
                     gpus = '0', 
                     callbacks = [checkpoint, bar], 
                     num_sanity_val_steps = 0,
                     auto_lr_find = True,
                     max_epochs = classify_epochs)

trainer.fit(classifier_model, 
            train_dataloader = loader[mode]['train'],
            val_dataloaders = loader[mode]['val'])

C:\Users\user\anaconda3\lib\site-packages\pytorch_lightning\utilities\device_parser.py:130: LightningDeprecationWarning: Parsing of the Trainer argument gpus='0' (string) will change in the future. In the current version of Lightning, this will select CUDA device with index 0, but from v1.5 it will select gpus [] (same as gpus=0 (int)).
  rank_zero_deprecation(
GPU available: True, used: True
TPU available: False, using: 0 TPU cores
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name  | Type            | Params
------------------------------------------
0 | model | YahooClassifier | 100 M 
------------------------------------------
100 M     Trainable params
0         Non-trainable params
100 M     Total params
400.002   Total estimated model params size (MB)
C:\Users\user\anaconda3\lib\site-packages\pytorch_lightning\trainer\data_loading.py:102: UserWarning: The dataloader, train dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `nu

Training: 0it [00:00, ?it/s]

C:\Users\user\anaconda3\lib\site-packages\pytorch_lightning\trainer\trainer.py:897: UserWarning: Detected KeyboardInterrupt, attempting graceful shutdown...
  rank_zero_warn('Detected KeyboardInterrupt, attempting graceful shutdown...')


In [ ]:
save_path = f"../Trained_Models/Classification/trained_model/trained_classification_model_{emb_dim}d_{classify_epochs}e"
torch.save(yahoo_classifier.state_dict(), save_path+'_latesum')

In [19]:
# save_path = f"../Trained_Models/Classification/trained_model/trained_classification_model_{emb_dim}d_{classify_epochs}e"
# classifier = YahooClassifier(word_embedding, emb_dim, class_size,device).to(device)
# yahoo_classifier.load_state_dict(torch.load(save_path))

In [20]:
trainer.test(test_dataloaders = loader[mode]['test'])

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
C:\Users\user\anaconda3\lib\site-packages\pytorch_lightning\trainer\data_loading.py:102: UserWarning: The dataloader, test dataloader 0, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 12 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(


Testing: 0it [00:00, ?it/s]

RuntimeError: CUDA error: device-side assert triggered
CUDA kernel errors might be asynchronously reported at some other API call,so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1.

In [ ]:
trainer.test(test_dataloaders = loader['msp']['test'])

In [ ]:
a = {True:1, False:2}
a

In [ ]:
a = [1,2,3]
b = [4,5,6]
c = [7,8,9]
ld = {
    'msp':{'train':a},
    'cor':{'train':c}
}
print(ld['msp']['train'])